<a href="https://colab.research.google.com/github/Germandev55/Topology_and_Geometry-for-Deep-Learning/blob/main/Betti_manifold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D

import IPython
from IPython.display import HTML
from google.colab import output

import tensorflow as tf
from keras.datasets import cifar10
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense, BatchNormalization, Dropout, Convolution2D, Input,Activation, ZeroPadding2D, MaxPooling2D, Flatten
from keras import regularizers

import sklearn
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.manifold import TSNE
from sklearn.manifold import LocallyLinearEmbedding
from sklearn import preprocessing
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
max_abs_scaler = preprocessing.MaxAbsScaler()

!pip install ripser
!pip install persim
!pip install umap-learn
from ripser import ripser
from ripser import Rips
from persim import plot_diagrams
import umap

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train * (1./255.)
x_test = x_test * (1./255.)

In [ ]:
def features_to_X(x_test, model, index_layer, dim_red = 2):
  input = x_test
  layer = model.get_layer(index=index_layer) 
  layer_name = str(layer).split('.')[5].split(' ')[0]
  features_layer1 = tf.keras.models.Model(inputs=model.inputs, outputs = layer.output)
  output_ = features_layer1(input) 
  try:
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    feature_batch_average = global_average_layer(output_)
    X = feature_batch_average   
  except:
    X = output_   
  return X, layer_name

In [ ]:
def get_Betti(data, eps, maxdim=2):
  Betti_layer = []
  dgms = ripser(data, maxdim=maxdim, thresh=eps)['dgms']  
  inf_in_list = lambda x: 1 if np.isinf(x).any() else 0
  Betti_list = [0,0,0]
  for e, i in enumerate(dgms):
    for x in i: Betti_list[e] += inf_in_list(x)
  return sum(Betti_list)

In [ ]:
def get_Betti_prob(data, eps, maxdim=2):
  dgms = ripser(data, maxdim=maxdim, thresh=eps)['dgms']  
  prev_max = 0
  list_Hom = []
  mean_Hom = []
  for Hom in dgms:
    B_Hom = 0
    for i in Hom:   
      if np.isinf(i).any() == False:
        B_Hom += (i[1] - i[0])
        prev_max = i[1]
      else:
        continue 
        B_Hom += prev_max
    list_Hom.append(B_Hom)
    l = len(Hom)
    if len(Hom) == 0: l=1
    mean_Hom.append(B_Hom/l)

  mean_Hom =  [round(x,3) for x in mean_Hom] 
  return sum(list_Hom), sum(mean_Hom)

In [ ]:
from sklearn.manifold import SpectralEmbedding
def Betti_generaliz(models,eps_lft,examples):
  models = models[0]
  Global_Betti_prob = []
  Global_Betti_mean = []
  Global_Betti_sum = []

  Betti_layer_listglobal = []
  Betti_layer_mean_listglobal = []
  sum_betti_listglobal = []

  for model in models:
    X_emb, layer_name = features_to_X(x_train[:examples], model, -2) 
    print(X_emb.shape[1])

    Xtest = min_max_scaler.fit_transform(X_emb)
    #Xtest = preprocessing.normalize(X_emb, norm='l1')
    
    #Xtest = max_abs_scaler.fit_transform(X_emb)

    #embedding = LocallyLinearEmbedding(n_neighbors=120, n_components=10)
    #Xtest  = embedding.fit_transform(Xtest)

    #UMAP_reducer = umap.UMAP(n_components=7, min_dist=0.7, n_neighbors=1400, metric='euclidean') # 0.7/50/euclidean
    #Xtest = UMAP_reducer.fit_transform(Xtest)

    #Xtest = X_emb 
    print('Betti compute')
    
    Betti_layer_class = []
    Betti_layer_mean_class = []

    for c in list(range(10)):
      X_emb_class = []
      for e, i in enumerate(y_train[:examples]):
        if i == c: X_emb_class.append(Xtest[e])
      Xtest_class = np.array(X_emb_class)

      Betti_layer, Betti_layer_mean = get_Betti_prob(Xtest_class, eps_lft, 2)

      Betti_layer_class.append(Betti_layer)
      Betti_layer_mean_class.append(Betti_layer_mean)

    Betti_layer_listglobal.append(sum(Betti_layer_class)/10)
    Betti_layer_mean_listglobal.append(sum(Betti_layer_mean_class)/10)  

  Betti_layer_listglobal = [round(x,5) for x in  Betti_layer_listglobal]
  Betti_layer_mean_listglobal = [round(x,5) for x in Betti_layer_mean_listglobal]
  
  return Betti_layer_listglobal, Betti_layer_mean_listglobal

In [ ]:
examples = 4000
eps_lft = np.inf
Betti_local, Betti_global = Betti_generaliz(models_resnet, eps_lft, examples)
print(Betti_local,Betti_global)